In [22]:
import os
import glob
import pandas as pd

directory = '/home/ulysses/Documents/Naerk/naerk-wiki/quartz/content/Gods/.data'

os.chdir(directory)
extension = 'tsv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

dataframes = []  # List to store the DataFrames

for csv_file in all_filenames:
    df = pd.read_csv(csv_file, sep='\t')
    dataframes.append(df)

merged_df = pd.merge(dataframes[0], dataframes[1], on='Name', how='outer')

for i in range(2, len(dataframes)):
    merged_df = pd.merge(merged_df, dataframes[i], on='Name', how='outer')

# older implementation:
# df = merged_df.set_index('Name').transpose().sort_index(ascending=True)
    
df = merged_df.T.reset_index()
df.columns = df.loc[0]
df = df.drop(0).sort_values(by="Name").reset_index(drop=True)
df.tail()



,Name,Anathema,Category,Cleric Spells,Divine Ability,Divine Font,Divine Skill,Domains,Edicts,Favored Weapon,Followers,Pantheon,Plane,Portfolio,Title
48,Voolthow,"Destroy Undead, Heal living",Fifth Dimension,"1st: Grim Tendrils, 3rd: Wall of Shadow, 5th: ...",Wisdom or Charisma,Harm,Deception,"Darkness, Nightmares, Pain, Sorrow, Undeath","Corrupt material plane, Create Undead",Flail,"Necromancers, nihilists, acolytes, shadow crea...",-,Shadow,"Undead, Corruption, Domination, Negative Emotions","Shadow Lord, ""Undead Master"", ""Corruptor"""
49,Xeurim,"Spoil a secret, Ignore an opportunity","Archdemon, Conspirators","1st: Illusory Disguise, 3rd: Sculpt Sound, 4th...",Dexterity or Charisma,Harm,Deception,"Ambition, Cities, Change, Darkness, Passion, S...","Deceive other fools, Manipulate persons of pow...",Dagger,"Thieves, Diplomats, Assassins, Pirates, Order ...",Demonic,Abyss,"Manipulation, Deception, Disguise, Murder, Sub...","Two-Faced, ""Lady of Lust"""
50,Yast,"Bind to objects, Wealth",Fifth Dimension,"1st: Grim Tendrils, 5th: Aberrant Form, 9th: A...",Intelligence or Charisma,Harm,Occultism,"Abomination, Darkness, Destruction, Nightmares...","Invade and destroy material plane, Grief, Hate",Bastard Sword,"lunatics, nihilists, aberrations, abominations",-,Void,"Destruction, Fear, Sorrow, Outer beings","Void, ""Outer Child"", ""Resentful"""
51,Yerastu,"Raise Undead, Break time or space, Steal soul",-,"1st: True Strike, 3rd: Ghostly Weapon, 9th: Su...",Intelligence or Wisdom,Heal or Harm,Religion,"Death, Duty, Fate, Protection, Repose, Soul, T...","Respect the Circle of Life, Do not disturb tim...",Longsword,"Pilgrims, Guardians, Sages, Baelarch, Gatewatc...",The Just,"Tailway, Fortress of the Just","Vigilance, Judgement of the dead, Punish those...",The Observer
52,Yushan,"Repetition, Complacency",Primordial,"1st: Illusory Disguise, 2nd: Mirror Image, 4th...",Intelligence or Wisdom,Heal or Harm,Arcana,"Change, Freedom, Knowledge, Luck, Magic, Time,...","Question Reality, Seek Paradox, Expose Contrad...",Bladed Scarf,"Seekers, inventors, outcasts, exiles, wanderer...",Vethian,Zinctizuctra,"Books, Writing, Experiments, Discoveries, Memo...","Master of Paradoxes, ""Inconsistent"", ""Liar Ete..."


### Export to excel

In [185]:
#%% EXPORT TO EXCEL

excelname = 'Gods.xlsx'
df.to_excel(excelname, sheet_name="Data", index=False)
# df.to_csv("_dataframe.csv")

columns_to_analyse = ['Category', 'Portfolio', 'Followers', 'Pantheon', 'Plane', 'Divine Skill', 'Favored Weapon', 'Domains']

for col in columns_to_analyse:
    explode = df[col].str.split(', ').explode()
    valuecounts = explode.value_counts()

    dfx = valuecounts.rename_axis(col).to_frame('count').reset_index()
    dfx['occurences'] = dfx[col].apply(lambda x: df[df[col].str.contains(x)]['Name'].tolist())
    with pd.ExcelWriter(excelname, engine='openpyxl', mode='a') as writer:
        dfx.to_excel(writer, sheet_name=col, index=False)

In [28]:
import numpy as np

columns_to_analyse = ['Category', 'Portfolio', 'Followers', 'Pantheon', 'Plane', 'Divine Skill', 'Favored Weapon', 'Domains']

for col in columns_to_analyse:
    explode = df[col].str.split(', ').explode()
    valuecounts = explode.value_counts()

    # dfx = valuecounts.rename_axis(col).to_frame('count').reset_index()
    # dfx['occurences'] = dfx[col].apply(lambda x: df[df[col].str.contains(x)]['Name'].tolist())
    explode.value_counts().plot.pie(kind='pie')


TypeError: <pandas.plotting._core.PlotAccessor object at 0x7f89053f02e0> got multiple values for keyword argument 'kind'

In [8]:
df.groupby(['Category','Name']).count()

0                                    Anathema  Cleric Spells  Divine Ability  \
Category                Name                                                   
-                       Amehan              1              1               1   
                        Hedbba              1              1               1   
                        Huntsman            1              1               1   
                        Karadosh            1              1               1   
                        Kromlek             1              1               1   
                        Nahema              1              1               1   
                        Reszek              1              1               1   
                        Sigard              1              1               1   
                        Steenar             1              1               1   
                        Vard                1              1               1   
                        Yerastu             1              1               1   
Archdemon, Conspirators Otynhash            1              1               1   
                        Xeurim              1              1               1   
Ascended                Anhalt              1              1               1   
                        Lainek              1              1               1   
                        Lleyn               1              1               1   
                        Sennek              1              1               1   
Aspects                 B'tolen             1              1               1   
                        Berohath            1              1               1   
                        Crel'vach           1              1               1   
                        Drokasner           1              1               1   
                        Fallfaunt           1              1               1   
                        Il'vec              1              1               1   
                        Jalerk              1              1               1   
                        Lastra              1              1               1   
                        Mormankar           1              1               1   
                        Sseratush           1              1               1   
                        Tarov               1              1               1   
                        Valanshore          1              1               1   
                        Volrek              1              1               1   
Chosen                  Disiklek            1              1               1   
Devils                  Sergonial           1              1               1   
                        Veiroch             1              1               1   
Dragon                  Klemnestron         1              1               1   
                        Nentha              1              1               1   
Fifth Dimension         Neverborn           1              1               1   
                        Voolthow            1              1               1   
                        Yast                1              1               1   
Human                   Hansa               1              1               1   
Luminous Walkers, Human Factar              1              1               1   
                        Laerion             1              1               1   
Primordial              Fyrlean             1              1               1   
                        Gibba               1              1               1   
                        Hydarax             1              1               1   
                        Klont               1              1               1   
                        Maeronian           1              1               1   
                        Malahir             1              1               1   
                        Saekla              1              1               1   
